PROBLEM STATUS: In Progress

PROBLEM DIFFICULTY LEVEL: 70%

PROBLEM STATEMENT
https://projecteuler.net/problem=282

I decided to try this as an Notebook as I feel there is benefit in visualisation and documenting my thought process. 
Solving some of these problems, as in real life, requires some level of intuition as well.

In [ ]:
#Importing libraries that might be important
import numpy as np

In [90]:
#We'll define a function for good practice
def ackerman_function(m,n):
    '''
    Description:
        Defines the Ackerman function and evaluates its value given two inputs m and n
    Args:
        m (int) and n (int): the arguments to the Ackerman function
    Returns:
        The value of the evaluated Ackerman function
    '''
    #It is clear that this function will continue to run itself until we reach a case where m has been reduced to 0
    #Defining the variable that holds the value of Ackerman function and initialising to 0
    
    function_evaluation = 0
    #Based on the definition, the Ackerman function is a piece wise function. Our first task is to define which case/branch of the function we are dealing with
    if(m==0):
        function_evaluation = n+1
        return(function_evaluation)
    elif((m>n) and (n==0)):
        #Recalling the ackerman function
        return(ackerman_function(m-1,1))
    else:
        #Recalling the ackerman function
        return(ackerman_function(m-1,ackerman_function(m,n-1)))

In [91]:
#Testing out our function for a few values of m and n
print(f"A(1,0) = {ackerman_function(1,0)}") #Expected to evaluate to 2
print(f"A(2,2) = {ackerman_function(2,2)}") #Expected to evaluate to 7
print(f"A(3,4) = {ackerman_function(3,4)}") #Expected to evaluate to 125
print(f"A(0,3) = {ackerman_function(0,3)}") #Expected to evaluate to 125

A(1,0) = 2
A(2,2) = 7
A(3,4) = 125
A(0,3) = 4


Now that we know that the function evaluates to the examples given in the problem definition, we know we've defined Ackerman function properly. 
The answer to this problem lies in the following Sum; A(0,0) + A(1,1) + A(2,2) + A(3,3) + A(4,4) + A(5,5) + A(6,6).

It would be great if we could perform a loop that solves each one however we quickly see that anything in the form A(n,n) for n>3 provides a recursion error as we are calling the function too often.

To prove it:

In [92]:
#We will set up a loop that evaluates A(n,n) until an exception is thrown.
for n in range (7):
    try:
        print(f"A({n},{n}) = {ackerman_function(n,n)}")
    except Exception as e:
        print(f'"{e}"- error thrown when trying to evaluate A({n},{n})')

A(0,0) = 1
A(1,1) = 3
A(2,2) = 7
A(3,3) = 61
"maximum recursion depth exceeded"- error thrown when trying to evaluate A(4,4)
"maximum recursion depth exceeded"- error thrown when trying to evaluate A(5,5)
"maximum recursion depth exceeded"- error thrown when trying to evaluate A(6,6)


This highlights that we'll need to simplify / decompose the Ackerman function to be able to break such large evaluations into smaller ones.
I'll start by exploring the arguments that are possible to evaluate, via an LxL matrix of evaluations, to gain a better picture of what we're dealing with.
I have a feeling we'll see a pattern emerge.

In [93]:
#Defining L*L array that will hold our values
l =25
our_matrix = np.empty((4,l))
#Let us set up a loop to store the arguments and evaluations
for m in range(4):
    for n in range(l):
        #We will try to evaluate the function
        try:
            our_matrix[m][n]=ackerman_function(m,n)
        except Exception as e:
            #If we encounter an error we will save as an nan value
            our_matrix[m][n]=np.nan
#I'll create a similar array as a pd to make it easier to visualise the combinations as pd will automatically give us the row and column numbers
#This is just for quick easy visualisaition for the time being and not for any calculation purposes
import pandas as pd
our_matrix_pd = pd.DataFrame(our_matrix)
print(our_matrix_pd)

    0     1     2     3      4      5      6       7       8     9   ...  \
0  1.0   2.0   3.0   4.0    5.0    6.0    7.0     8.0     9.0  10.0  ...   
1  2.0   3.0   4.0   5.0    6.0    7.0    8.0     9.0    10.0  11.0  ...   
2  3.0   5.0   7.0   9.0   11.0   13.0   15.0    17.0    19.0  21.0  ...   
3  5.0  13.0  29.0  61.0  125.0  253.0  509.0  1021.0  2045.0   NaN  ...   

     15    16    17    18    19    20    21    22    23    24  
0  16.0  17.0  18.0  19.0  20.0  21.0  22.0  23.0  24.0  25.0  
1  17.0  18.0  19.0  20.0  21.0  22.0  23.0  24.0  25.0  26.0  
2  33.0  35.0  37.0  39.0  41.0  43.0  45.0  47.0  49.0  51.0  
3   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  

[4 rows x 25 columns]


We can immidiately see that there is a pattern, and luckily a decomposition in addition. For example:
<br>A(3,2) = 29
<br>and
<br>A(3,1) + A(2,4) + A(0,4) = 13 + 11 + 5 = 29 = A(3,2)

Namely, *any evaluated Ackerman can be defined as the _sum_ of one or more other evaluated Ackerman*. As another example:
<br>A(3,4) = 125
<br>and
<br>A(3,3) + A(3,2) + A(3,1) + A(2,4) + A(2,2) + A(2,0) + A(0,0) = 61 + 29 + 13 + 11 + 7 + 3 + 1 = 125

This breakthrough shows us that, at least computationally until the limits we'eve so far defined, any Ackerman function (except A(0,0), A(1,0) and A(0,1)) can be broken down into the sum of two or more *unique* Ackerman functions. Equipped with this, we can now continue to dig deeper to uncover the underlying formula that efficintly describes the composition. 

Because we now have a list of 

**Note: We also see that along the way we will have to deal with the fact that there may be multiple Ackerman functions that evaluate to the same number, hence we will also need to find which of these arguments works best for our given input.**

Lets simply our findings to the A(2,3) case and see if we can create another breakthrough. It is sizeable enough to give a few possibilities, but not large enough where the possibilities are endless.

We will start by defining an LxL matrix that gets as far as